In [173]:
from __future__ import annotations
import zipfile
import pandas as pd
import os
from io import StringIO
import matplotlib.pyplot as plt
import tempfile
import plotly.express as px


In [174]:
DATA_ZIP = "/Users/alex/Desktop/CS/Internships/DBF/python_tools/data/windTunnel/ALL/windtunnel_data.zip"
OUTPUT_DIR = "/Users/alex/Desktop/CS/Internships/DBF/python_tools/data/windTunnel/ALL/"

In [175]:
df = pd.DataFrame(columns=["Motor", "Prop", "Airspeed", "Cells", "Battery", "Config", "Thrust"])

with zipfile.ZipFile(DATA_ZIP) as z:
    for name in filter(lambda x: x.endswith(".csv") and not x.startswith('__MACOSX'), z.namelist()):
        motor, prop, airspeed, cells, battery, config = name.removeprefix("windtunnel_data/").removesuffix(".csv").split("_")
        battery = battery.lower()
        data = z.read(name)
        _df = pd.read_csv(StringIO(data.decode('utf-8')))
        print(name)
        if config != "PT":
            _df = _df[_df["CODE"] == 0]
            _df.reset_index(inplace=True, drop=True)
        thrust = _df["Drag"].min() * -1
        
        df.loc[-1] = [motor, prop.lower(), airspeed, cells, battery, config, thrust]
        df.index = df.index + 1

windtunnel_data/AT7215_16x14_30ms_12S_NaN_PT.csv
windtunnel_data/AT7215_15x14x3_45ms_12S_115Wh_M2.csv
windtunnel_data/AT7215_15x14x3_30ms_12S_97.5Wh_M3.csv
windtunnel_data/AT7215_16x14_50ms_12S_2.8Ah_M2.csv
windtunnel_data/AT7215_16x14_45ms_12S_NaN_MAX.csv
windtunnel_data/AT5330_16x14_20ms_12S_NaN_PT.csv
windtunnel_data/AT7215_15x14x3_40ms_12S_2.4Ah_DT.csv
windtunnel_data/AT7215_16x14_35ms_12S_2.4Ah_M3.csv
windtunnel_data/AT7215_16x14_45ms_12S_NaN_PT.csv
windtunnel_data/AT7215_16x14_20ms_12S_NaN_PT.csv
windtunnel_data/AT7215_16x14_10ms_12S_115Wh_RUN1.csv
windtunnel_data/AT5220_15X14X3_40ms_8S_2.4aH_RUN1.csv
windtunnel_data/AT5330_16x16_30ms_12S_NaN_PT.csv
windtunnel_data/AT7215_15x14x3_35ms_12S_97.5Wh_M3.csv
windtunnel_data/AT5220_16X12_30ms_8S_NaN_RUN1.csv
windtunnel_data/AT7215_16x14_15ms_12S_NaN_PT.csv
windtunnel_data/AT7215_16x16_45ms_12S_NaN_PT.csv
windtunnel_data/AT5220_15X14X3_35ms_8S_2.0aH_RUN1.csv
windtunnel_data/AT7215_16x14_40ms_12S_97.5Wh_M3.csv
windtunnel_data/AT5330_16x14

In [176]:
df = df[df["Airspeed"] != "NaN"]

In [177]:
vcounts = df.value_counts()
vcounts.to_csv(os.path.join(OUTPUT_DIR, "vcounts.csv"))

In [178]:
# find rows with duplicated "Motor", "Prop", "Airspeed", "Cells" and print the battery, config and thrust
duplicated = df[df.duplicated(subset=["Motor", "Prop", "Airspeed", "Cells"], keep=False)]
duplicated[["Motor", "Prop", "Airspeed", "Cells", "Battery", "Config", "Thrust"]].groupby(["Motor", "Prop", "Airspeed", "Cells"]).agg(list)

Battery  \
Motor  Prop    Airspeed Cells                                       
AT5220 15x14x3 35ms     8S                         [2.0ah, 3.2ah]   
AT7215 15x14x3 10ms     12S                         [97.5wh, nan]   
               30ms     12S                  [97.5wh, nan, 2.4ah]   
               35ms     12S           [97.5wh, 2.4ah, nan, 2.4ah]   
               40ms     12S    [2.4ah, 115wh, 2.4ah, 97.5wh, nan]   
               45ms     12S            [115wh, 2.4ah, nan, 2.8ah]   
               50ms     12S                          [nan, 2.8ah]   
               static   12S                         [nan, 97.5wh]   
       16x14   10ms     12S                          [115wh, nan]   
               30ms     12S                  [nan, 2.4ah, 97.5wh]   
               35ms     12S                  [2.4ah, nan, 97.5wh]   
               40ms     12S           [97.5wh, nan, 2.4ah, 115wh]   
               45ms     12S       [nan, nan, 2.8ah, 2.8ah, 115wh]   
               50ms     12S              [2.8ah, nan, 2.8ah, nan]   
               55ms     12S              [2.8ah, nan, 2.8ah, nan]   
               static   12S                  [115wh, nan, 97.5wh]   

                                              Config  \
Motor  Prop    Airspeed Cells                          
AT5220 15x14x3 35ms     8S              [RUN1, RUN1]   
AT7215 15x14x3 10ms     12S               [RUN1, PT]   
               30ms     12S             [M3, PT, M3]   
               35ms     12S         [M3, DT, PT, M3]   
               40ms     12S     [DT, M2, M3, M3, PT]   
               45ms     12S         [M2, DT, PT, M2]   
               50ms     12S                 [PT, M2]   
               static   12S               [PT, RUN1]   
       16x14   10ms     12S               [RUN1, PT]   
               30ms     12S             [PT, M3, M3]   
               35ms     12S             [M3, PT, M3]   
               40ms     12S         [M3, PT, M3, M2]   
               45ms     12S    [MAX, PT, M2, DT, M2]   
               50ms     12S        [M2, PT, DT, MAX]   
               55ms     12S        [M2, M2, DT, MAX]   
               static   12S         [RUN1, PT, RUN1]   

                                                                 Thrust  
Motor  Prop    Airspeed Cells                                            
AT5220 15x14x3 35ms     8S                                [13.1, 14.77]  
AT7215 15x14x3 10ms     12S                             [23.37, 27.702]  
               30ms     12S                      [7.195, 21.683, 7.233]  
               35ms     12S             [8.039, 20.977, 20.938, 17.632]  
               40ms     12S      [19.384, 13.304, 17.23, 6.403, 18.317]  
               45ms     12S             [12.82, 16.849, 16.533, 17.067]  
               50ms     12S                            [13.443, 15.073]  
               static   12S                             [26.239, 23.61]  
       16x14   10ms     12S                            [17.386, 24.399]  
               30ms     12S                     [23.756, 21.926, 6.404]  
               35ms     12S                     [21.075, 23.357, 6.225]  
               40ms     12S             [6.281, 21.648, 19.326, 13.901]  
               45ms     12S    [17.585, 21.368, 19.659, 16.727, 14.432]  
               50ms     12S            [19.209, 18.339, 16.059, 16.346]  
               55ms     12S               [17.814, nan, 15.562, 16.965]  
               static   12S                    [18.291, 25.753, 20.616]

In [179]:
duplicated[duplicated["Battery"] != "NaN"].groupby(["Motor", "Prop", "Airspeed", "Cells"]).agg(list)

Battery  \
Motor  Prop    Airspeed Cells                                       
AT5220 15x14x3 35ms     8S                         [2.0ah, 3.2ah]   
AT7215 15x14x3 10ms     12S                         [97.5wh, nan]   
               30ms     12S                  [97.5wh, nan, 2.4ah]   
               35ms     12S           [97.5wh, 2.4ah, nan, 2.4ah]   
               40ms     12S    [2.4ah, 115wh, 2.4ah, 97.5wh, nan]   
               45ms     12S            [115wh, 2.4ah, nan, 2.8ah]   
               50ms     12S                          [nan, 2.8ah]   
               static   12S                         [nan, 97.5wh]   
       16x14   10ms     12S                          [115wh, nan]   
               30ms     12S                  [nan, 2.4ah, 97.5wh]   
               35ms     12S                  [2.4ah, nan, 97.5wh]   
               40ms     12S           [97.5wh, nan, 2.4ah, 115wh]   
               45ms     12S       [nan, nan, 2.8ah, 2.8ah, 115wh]   
               50ms     12S              [2.8ah, nan, 2.8ah, nan]   
               55ms     12S              [2.8ah, nan, 2.8ah, nan]   
               static   12S                  [115wh, nan, 97.5wh]   

                                              Config  \
Motor  Prop    Airspeed Cells                          
AT5220 15x14x3 35ms     8S              [RUN1, RUN1]   
AT7215 15x14x3 10ms     12S               [RUN1, PT]   
               30ms     12S             [M3, PT, M3]   
               35ms     12S         [M3, DT, PT, M3]   
               40ms     12S     [DT, M2, M3, M3, PT]   
               45ms     12S         [M2, DT, PT, M2]   
               50ms     12S                 [PT, M2]   
               static   12S               [PT, RUN1]   
       16x14   10ms     12S               [RUN1, PT]   
               30ms     12S             [PT, M3, M3]   
               35ms     12S             [M3, PT, M3]   
               40ms     12S         [M3, PT, M3, M2]   
               45ms     12S    [MAX, PT, M2, DT, M2]   
               50ms     12S        [M2, PT, DT, MAX]   
               55ms     12S        [M2, M2, DT, MAX]   
               static   12S         [RUN1, PT, RUN1]   

                                                                 Thrust  
Motor  Prop    Airspeed Cells                                            
AT5220 15x14x3 35ms     8S                                [13.1, 14.77]  
AT7215 15x14x3 10ms     12S                             [23.37, 27.702]  
               30ms     12S                      [7.195, 21.683, 7.233]  
               35ms     12S             [8.039, 20.977, 20.938, 17.632]  
               40ms     12S      [19.384, 13.304, 17.23, 6.403, 18.317]  
               45ms     12S             [12.82, 16.849, 16.533, 17.067]  
               50ms     12S                            [13.443, 15.073]  
               static   12S                             [26.239, 23.61]  
       16x14   10ms     12S                            [17.386, 24.399]  
               30ms     12S                     [23.756, 21.926, 6.404]  
               35ms     12S                     [21.075, 23.357, 6.225]  
               40ms     12S             [6.281, 21.648, 19.326, 13.901]  
               45ms     12S    [17.585, 21.368, 19.659, 16.727, 14.432]  
               50ms     12S            [19.209, 18.339, 16.059, 16.346]  
               55ms     12S               [17.814, nan, 15.562, 16.965]  
               static   12S                    [18.291, 25.753, 20.616]

In [180]:
df[df["Config"] == "M3"].sort_values(by=["Motor", "Prop", "Airspeed", "Cells"]).reset_index(drop=True)

Motor     Prop Airspeed Cells Battery Config  Thrust
0   AT7215  15x14x3     30ms   12S  97.5wh     M3   7.195
1   AT7215  15x14x3     30ms   12S   2.4ah     M3   7.233
2   AT7215  15x14x3     35ms   12S  97.5wh     M3   8.039
3   AT7215  15x14x3     35ms   12S   2.4ah     M3  17.632
4   AT7215  15x14x3     40ms   12S   2.4ah     M3  17.230
5   AT7215  15x14x3     40ms   12S  97.5wh     M3   6.403
6   AT7215    16x14     30ms   12S   2.4ah     M3  21.926
7   AT7215    16x14     30ms   12S  97.5wh     M3   6.404
8   AT7215    16x14     35ms   12S   2.4ah     M3  21.075
9   AT7215    16x14     35ms   12S  97.5wh     M3   6.225
10  AT7215    16x14     40ms   12S  97.5wh     M3   6.281
11  AT7215    16x14     40ms   12S   2.4ah     M3  19.326

In [181]:
df[df["Config"] == "M2"].sort_values(by=["Motor", "Prop", "Airspeed", "Cells"]).reset_index(drop=True)

Motor     Prop Airspeed Cells Battery Config  Thrust
0  AT7215  15x14x3     40ms   12S   115wh     M2  13.304
1  AT7215  15x14x3     45ms   12S   115wh     M2  12.820
2  AT7215  15x14x3     45ms   12S   2.8ah     M2  17.067
3  AT7215  15x14x3     50ms   12S   2.8ah     M2  15.073
4  AT7215    16x14     40ms   12S   115wh     M2  13.901
5  AT7215    16x14     45ms   12S   2.8ah     M2  19.659
6  AT7215    16x14     45ms   12S   115wh     M2  14.432
7  AT7215    16x14     50ms   12S   2.8ah     M2  19.209
8  AT7215    16x14     55ms   12S   2.8ah     M2  17.814
9  AT7215    16x14     55ms   12S     nan     M2     NaN

In [182]:
# keep only the max thrust for each "Motor", "Prop", "Airspeed", "Cells"
df_max = df.groupby(["Motor", "Prop", "Airspeed", "Cells"]).agg({"Thrust": "max"}).reset_index()
df_max

Motor     Prop Airspeed Cells  Thrust
0   AT5220  15x14x3     25ms    8S  15.538
1   AT5220  15x14x3     30ms    8S  14.885
2   AT5220  15x14x3     35ms    8S  14.770
3   AT5220  15x14x3     40ms    8S  10.570
4   AT5220  15x14x3   static    8S  18.135
5   AT5220    16x12     25ms    8S  15.762
6   AT5220    16x12     30ms    8S  14.680
7   AT5220    16x12     35ms    8S  13.679
8   AT5220    16x12     40ms    8S  13.343
9   AT5220    16x12   static    8S  16.754
10  AT5330  15x14x3     10ms   12S  19.193
11  AT5330  15x14x3     15ms   12S  16.997
12  AT5330  15x14x3     20ms   12S  16.177
13  AT5330  15x14x3     25ms   12S  15.198
14  AT5330  15x14x3     30ms   12S  12.796
15  AT5330  15x14x3     35ms   12S  11.208
16  AT5330  15x14x3   static   12S  19.676
17  AT5330    16x14     20ms   12S  18.028
18  AT5330    16x14     25ms   12S  17.511
19  AT5330    16x14     30ms   12S  16.379
20  AT5330    16x14     35ms   12S  15.358
21  AT5330    16x16     30ms   12S  17.130
22  AT5330    16x16     35ms   12S  15.961
23  AT5330    16x16     40ms   12S  15.205
24  AT7215  15x14x3     10ms   12S  27.702
25  AT7215  15x14x3     15ms   12S  25.014
26  AT7215  15x14x3     20ms   12S  25.089
27  AT7215  15x14x3     25ms   12S  23.968
28  AT7215  15x14x3     30ms   12S  21.683
29  AT7215  15x14x3     35ms   12S  20.977
30  AT7215  15x14x3     40ms   12S  19.384
31  AT7215  15x14x3     45ms   12S  17.067
32  AT7215  15x14x3     50ms   12S  15.073
33  AT7215  15x14x3   static   12S  26.239
34  AT7215    16x14     10ms   12S  24.399
35  AT7215    16x14     15ms   12S  22.907
36  AT7215    16x14     20ms   12S  24.525
37  AT7215    16x14     25ms   12S  24.706
38  AT7215    16x14     30ms   12S  23.756
39  AT7215    16x14     35ms   12S  23.357
40  AT7215    16x14     40ms   12S  21.648
41  AT7215    16x14     45ms   12S  21.368
42  AT7215    16x14     50ms   12S  19.209
43  AT7215    16x14     55ms   12S  17.814
44  AT7215    16x14     60ms   12S  14.999
45  AT7215    16x14   static   12S  25.753
46  AT7215    16x16     45ms   12S  20.074
47  AT7215    16x16     50ms   12S  20.357

In [183]:
df_max["Airspeed"] = df_max["Airspeed"].apply(lambda x: float(x.replace("static", "0.0").replace("ms", "")))

In [184]:
df_max_8s = df_max[df_max["Cells"] == "8S"]
df_max_12s = df_max[df_max["Cells"] == "12S"]

In [185]:
for motor in df_max_8s["Motor"].unique():
    df_motor = df_max_8s[df_max_8s["Motor"] == motor].sort_values(by="Airspeed")
    fig = px.scatter(df_motor, x="Airspeed", y="Thrust", color="Prop", title=motor + " 8S")
    fig.update_traces(mode='lines+markers')
    fig.update_layout(
        xaxis_title="Airspeed (m/s)",
        yaxis_title="Thrust (lbf)",
        legend_title="Prop",
    )
    fig.add_vrect(
        x0=35, x1=55,
        fillcolor="LightSalmon", opacity=0.5,
        layer="below", line_width=0,
    )
    fig.add_annotation(
        x=(55 + 35)/2, y=df_motor["Thrust"].max(),
        text="M2",
        showarrow=False,
        font=dict(
            size=16,
            color="red"
        ),
    )
    fig.add_vrect(
        x0=20, x1=30,
        fillcolor="LightSeagreen", opacity=0.5,
        layer="below", line_width=0,
    )
    fig.add_annotation(
        x=(30 + 20)/2, y=df_motor["Thrust"].max(),
        text="M3",
        showarrow=False,
        font=dict(
            size=16,
            color="seagreen"
        ),
    )
    fig.show()
    fig.write_image(os.path.join(OUTPUT_DIR, f"{motor}_8s.png"), width=1000, height=500)

In [186]:
for motor in df_max_12s["Motor"].unique():
    df_motor = df_max_12s[df_max_12s["Motor"] == motor].sort_values(by="Airspeed")
    fig = px.scatter(df_motor, x="Airspeed", y="Thrust", color="Prop", title=motor + " 12S")
    fig.update_traces(mode='lines+markers')
    fig.update_layout(
        xaxis_title="Airspeed (m/s)",
        yaxis_title="Thrust (lbf)",
        legend_title="Prop",
    )
    fig.add_vrect(
        x0=35, x1=55,
        fillcolor="LightSalmon", opacity=0.5,
        layer="below", line_width=0,
    )
    fig.add_annotation(
        x=(55 + 35)/2, y=df_motor["Thrust"].max(),
        text="M2",
        showarrow=False,
        font=dict(
            size=16,
            color="red"
        ),
    )
    fig.add_vrect(
        x0=20, x1=30,
        fillcolor="LightSeagreen", opacity=0.5,
        layer="below", line_width=0,
    )
    fig.add_annotation(
        x=(30 + 20)/2, y=df_motor["Thrust"].max(),
        text="M3",
        showarrow=False,
        font=dict(
            size=16,
            color="seagreen"
        ),
    )
    fig.show()
    fig.write_image(os.path.join(OUTPUT_DIR, f"{motor}_12s.png"), width=1000, height=500)

In [187]:
df['Battery'].unique()

array(['nan', '115wh', '97.5wh', '2.8ah', '2.4ah', '2.0ah', '3.2ah'],
      dtype=object)

In [195]:
df[(df["Config"] == 'M2') & df['Battery'].apply(lambda x: not x in ['nan', '115wh', '97.5wh'])].sort_values(by=['Prop', 'Airspeed'], ascending=True).reset_index(drop=True).to_csv(os.path.join(OUTPUT_DIR, "M2.csv"))

In [196]:
df[(df["Config"] == 'M3') & df['Battery'].apply(lambda x: not x in ['nan', '115wh', '97.5wh'])].sort_values(by=['Prop', 'Airspeed'], ascending=True).reset_index(drop=True).to_csv(os.path.join(OUTPUT_DIR, "M3.csv"))

In [208]:
pd.read_csv(os.path.join(OUTPUT_DIR, "M2.csv"), index_col=0)

Motor     Prop Airspeed Cells  Battery Config  Average Thrust
0  AT7215  15x14x3     45ms   12S  2200mAh     M2       15.163871
1  AT7215  15x14x3     50ms   12S  2200mAh     M2       13.173614
2  AT7215    16x14     45ms   12S  2200mAh     M2       16.208721
3  AT7215    16x14     50ms   12S  2200mAh     M2       15.790526
4  AT7215    16x14     55ms   12S  2200mAh     M2       15.157545

In [209]:
pd.read_csv(os.path.join(OUTPUT_DIR, "M3.csv"), index_col=0)

Motor     Prop Airspeed Cells  Battery Config  Average Thrust
0  AT7215  15x14x3     30ms   12S  1600mAh     M3        6.426517
1  AT7215  15x14x3     35ms   12S  1600mAh     M3        6.569484
2  AT7215  15x14x3     40ms   12S  1600mAh     M3        7.101650
3  AT7215    16x14     30ms   12S  1600mAh     M3        6.694281
4  AT7215    16x14     35ms   12S  1600mAh     M3        6.583793
5  AT7215    16x14     40ms   12S  1600mAh     M3        6.533040

In [212]:
df_max = df[df['Config'].apply(lambda x: x in ['DT', 'M2']) & df['Battery'].apply(lambda x: not x in ['nan', '115wh', '97.5wh'])].sort_values(by=['Prop', 'Airspeed'], ascending=True).reset_index(drop=True)
df_max

Motor     Prop Airspeed Cells Battery Config  Thrust
0   AT7215  15x14x3     35ms   12S   2.4ah     DT  20.977
1   AT7215  15x14x3     40ms   12S   2.4ah     DT  19.384
2   AT7215  15x14x3     45ms   12S   2.4ah     DT  16.849
3   AT7215  15x14x3     45ms   12S   2.8ah     M2  17.067
4   AT7215  15x14x3     50ms   12S   2.8ah     M2  15.073
5   AT7215    16x14     45ms   12S   2.8ah     M2  19.659
6   AT7215    16x14     45ms   12S   2.8ah     DT  16.727
7   AT7215    16x14     50ms   12S   2.8ah     M2  19.209
8   AT7215    16x14     50ms   12S   2.8ah     DT  16.059
9   AT7215    16x14     55ms   12S   2.8ah     M2  17.814
10  AT7215    16x14     55ms   12S   2.8ah     DT  15.562
11  AT7215    16x14     60ms   12S   2.8ah     DT  14.999

In [213]:
df_max["Airspeed"] = df_max["Airspeed"].apply(lambda x: float(x.replace("static", "0.0").replace("ms", "")))

In [214]:
# keep Config == DT for duplicated "Motor", "Prop", "Airspeed", "Battery", "Cells" combos
df_max = df_max.groupby(["Motor", "Prop", "Airspeed", "Battery", "Cells"]).agg({"Thrust": "min"}).reset_index()
df_max.replace('2.8ah', '2200mAh', inplace=True)
df_max.replace('2.4ah', '1600mAh', inplace=True)
df_max

Motor     Prop  Airspeed  Battery Cells  Thrust
0  AT7215  15x14x3      35.0  1600mAh   12S  20.977
1  AT7215  15x14x3      40.0  1600mAh   12S  19.384
2  AT7215  15x14x3      45.0  1600mAh   12S  16.849
3  AT7215  15x14x3      45.0  2200mAh   12S  17.067
4  AT7215  15x14x3      50.0  2200mAh   12S  15.073
5  AT7215    16x14      45.0  2200mAh   12S  16.727
6  AT7215    16x14      50.0  2200mAh   12S  16.059
7  AT7215    16x14      55.0  2200mAh   12S  15.562
8  AT7215    16x14      60.0  2200mAh   12S  14.999

In [215]:
df_max_8s = df_max[df_max["Cells"] == "8S"]
df_max_12s = df_max[df_max["Cells"] == "12S"]

In [216]:
for motor in df_max_12s["Motor"].unique():
    df_motor = df_max_12s[df_max_12s["Motor"] == motor].sort_values(by="Airspeed")
    df_motor['Battery/Prop'] = df_motor['Battery'] + "/" + df_motor['Prop']
    fig = px.scatter(df_motor, x="Airspeed", y="Thrust", color='Battery/Prop', title=motor + " 12S")
    fig.update_traces(mode='lines+markers')
    fig.update_layout(
        xaxis_title="Airspeed (m/s)",
        yaxis_title="Thrust (lbf)",
        legend_title="Prop",
    )
    fig.add_vrect(
        x0=40, x1=60,
        fillcolor="LightSalmon", opacity=0.5,
        layer="below", line_width=0,
    )
    fig.add_annotation(
        x=(60 + 40)/2, y=df_motor["Thrust"].max(),
        text="M2",
        showarrow=False,
        font=dict(
            size=16,
            color="red"
        ),
    )
    fig.add_vrect(
        x0=30, x1=40,
        fillcolor="LightSeagreen", opacity=0.5,
        layer="below", line_width=0,
    )
    fig.add_annotation(
        x=(40 + 30)/2, y=df_motor["Thrust"].max(),
        text="M3",
        showarrow=False,
        font=dict(
            size=16,
            color="seagreen"
        ),
    )
    fig.show()
    fig.write_image(os.path.join(OUTPUT_DIR, f"{motor}_12s_new.png"), width=1000, height=500)